<a href="https://colab.research.google.com/github/ProfeKing/NSPH_Project/blob/main/NSPH_CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from google.colab import output
output.enable_custom_widget_manager()

In [1]:
from google.colab import files
uploaded = files.upload()

Saving heart.csv to heart.csv


In [ ]:
!pip install catboost
!pip install scikit-learn
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import accuracy_score

In [6]:
heart = pd.read_csv("heart.csv")

In [11]:
heart.head(1)

,age,sex,cp,trestbps,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,chol_lvl
0,52,1,0,125,0,1,168,0,1.0,2,2,3,0,2


In [9]:
'''nuova tabella di colesterolo con valori pari a 1 2 e 3'''
def categorise(row):  
    if row['chol'] <= 200:
        return 1
    elif row['chol'] > 200 and row['chol'] <= 239:
        return 2
    elif row['chol'] >= 240:
        return 3

heart['chol_lvl'] = heart.apply(lambda row: categorise(row), axis=1)

In [10]:
del(heart["chol"])

In [12]:
y = heart.target

In [14]:
X = heart.drop("target", axis=1)

In [32]:
features = ["age","exang", "trestbps","thalach", "oldpeak"]   
X = X[features]

In [39]:
X

,age,exang,trestbps,thalach,oldpeak
0,52,0,125,168,1.0
1,53,1,140,155,3.1
2,70,1,145,125,2.6
3,61,0,148,161,0.0
4,62,0,138,106,1.9
...,...,...,...,...,...
1020,59,1,140,164,0.0
1021,60,1,125,141,2.8
1022,47,1,110,118,1.0
1023,50,0,110,159,0.0


In [33]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.75, random_state=42)

In [40]:
model = CatBoostClassifier(
    custom_loss=[metrics.Accuracy()],
    random_seed=42,
    logging_level='Silent'
)

In [41]:
model.fit(
    X_train, y_train,
    eval_set=(X_validation, y_validation),
#     logging_level='Verbose',  # you can uncomment this for text output
    plot=True
);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [36]:
cv_params = model.get_params()
cv_params.update({
    'loss_function': metrics.Logloss()
})
cv_data = cv(
    Pool(X, y),
    cv_params,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [37]:
print('Best validation accuracy score: {:.2f}±{:.2f} on step {}'.format(
    np.max(cv_data['test-Accuracy-mean']),
    cv_data['test-Accuracy-std'][np.argmax(cv_data['test-Accuracy-mean'])],
    np.argmax(cv_data['test-Accuracy-mean'])
))

Best validation accuracy score: 0.98±0.02 on step 655


In [38]:
print('Precise validation accuracy score: {}'.format(np.max(cv_data['test-Accuracy-mean'])))

Precise validation accuracy score: 0.9755620723362659
